# SAN JOSE
## from open street map 

San Jose is one of favorites area in California. San Jose has many taste food. I always to there to find interesting and new restaurant. I want to use this data to find out something interesting about this are. This is the reason I choose this area as my project. 

In [1]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

#OSM_FILE = "C:\Users\Administrator\udacity\data analyst\P03\honolulu_hawaii.osm\honolulu_hawaii.osm"  # Replace this with your osm file
#OSM_FILE='C:\Users\Administrator\udacity\data analyst\P03\san-francisco-bay_california.osm\san-francisco-bay_california.osm'
OSM_FILE= 'C:\\Users\\Administrator\\udacity\\data analyst\\P03\\san-jose_california.osm\\san-jose_california.osm'
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

In [2]:
osm_file='C:\\Users\\Administrator\\udacity\\data analyst\\P03\\san-jose_california.osm\\san-jose_california.osm'


In [3]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET
from collections import OrderedDict
from collections import defaultdict
import pprint
import string


In [4]:
LOWER = re.compile(r'^([a-z]|_)*$')

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [5]:
def get_element(filename):
    """ Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = ET.iterparse(filename, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end':
            yield elem
            root.clear()

def count_tags(filename):
    tags = {}
    for i, elem in enumerate(get_element(filename)):
        if elem.tag in tags:
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags

# {'member': 1322,
#  'nd': 149685,
#  'node': 127664,
#  'osm': 1,
#  'relation': 124,
#  'tag': 68778,
#  'way': 16971}
    

### showing how many entires we have in different tags

In [6]:
count_tags(osm_file)

{'bounds': 1,
 'member': 13767,
 'nd': 1491733,
 'node': 1276635,
 'osm': 1,
 'relation': 1238,
 'tag': 685554,
 'way': 169712}

In [7]:
def key_type(filename):
    """ 
    Count the criteria in dictionary for the content of the tag.
    """
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for i, element in enumerate(get_element(filename)):
        if element.tag == "tag":
            if PROBLEMCHARS.search(element.attrib['k']):
                keys['problemchars']+=1
            elif LOWER.search(element.attrib['k']):
                keys['lower'] +=1
            elif LOWER_COLON.search(element.attrib['k']):
                keys['lower_colon']+=1
            else:
                keys['other']+=1
        
    return keys


#check
#other = include capital letter

    

### counting for different type of tags that we have.

In [8]:
key_type(osm_file)

{'lower': 371667, 'lower_colon': 298371, 'other': 15515, 'problemchars': 1}

In [9]:
def get_user(filename):
    users = set()
    for i,element in enumerate(get_element(filename)):
        if "uid" in element.attrib:
            users.add(element.attrib["uid"])
    return "Total users : " ,len(users)

### total user 

In [10]:
get_user(osm_file)

('Total users : ', 1194)

In [11]:
def tag_list(filename,item):
    tag_type = defaultdict(set)
    tag_type["regular"]=0
    colon=0
    not_colon=0
    for i, elem in enumerate(get_element(filename)):
        if elem.tag==item:
            for tags in elem.iter('tag'): 
                if not PROBLEMCHARS.search(tags.attrib['k']):
                    if LOWER_COLON.search(tags.attrib['k']):
                        colon+=1
                        sub_attrib=tags.attrib['k'].split(':',1)

                        if sub_attrib[0] in tag_type:
                            tag_type[sub_attrib[0]]+=1
                        else:
                            tag_type[sub_attrib[0]]=1
                    else: 
                        not_colon+=1
                        tag_type["regular"]+=1
    return {'colon':colon, 'not_colon':not_colon ,'tag_type_list': tag_type}

def count_tag_type(filename):

    tag_item=['member','nd','node','osm','relation','way']

    for item in tag_item:

        a=item
        result=tag_list(filename,item)
        if result['colon']+result['not_colon']<=0:
            print a,"　:　"
            print a ," do not have tag"
            print "<------------------------------------------------------------------------>"
        else:
            print a
            print "number of items with colon: ",result['colon']
            print "number of items without colon: ", result['not_colon']
            print "list of tag type in ", a, " :", result['tag_type_list'].items()
            print "<------------------------------------------------------------------------>"
                 

    return None

    

### counting each type in different tags
Base on this information, we choose the type that we want to check.

In [12]:
count_tag_type(osm_file)

member 　:　
member  do not have tag
<------------------------------------------------------------------------>
nd 　:　
nd  do not have tag
<------------------------------------------------------------------------>
node
number of items with colon:  24145
number of items without colon:  42175
list of tag type in  node  : [('shop', 20), ('amenity', 37), ('telescope', 19), ('proposed', 2), ('recycling', 14), ('social_facility', 2), ('wetap', 5), ('alt_name', 4), ('gnis', 2229), ('toilets', 10), ('exit_to', 18), ('capacity', 1), ('addr', 20176), ('service', 7), ('name', 115), ('source', 254), ('railway', 7), ('fuel', 17), ('place', 1), ('is_in', 46), ('ref', 14), ('website', 7), ('caltrans', 3), ('description', 1), ('theatre', 1), ('fire_hydrant', 25), ('office', 1), ('stop', 1), ('townhall', 1), ('leisure', 1), ('outerspatial', 2), ('tiger', 47), ('flag', 14), ('regular', 42175), ('award', 1), ('socket', 11), ('old_name', 11), ('reuse', 1), ('monitoring', 5), ('payment', 37), ('population', 

# we are going to check street name and postcode
we aduit the data that we choose, and display the items that is not what we expected.

In [13]:
street_type_re=re.compile(r'\b\S+\.?$', re.IGNORECASE)
zipcode_re =re.compile(r"\d{5}?")
no_number_re=re.compile(r'(?=(\w*\s?\w+\s\w+)\s+[Ste|Suite|PM])', re.IGNORECASE)
street_name_only_re= re.compile(r"\A[\w\s]*", re.IGNORECASE)
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Way", "Terrace", "Circle", "Real", "Expressway", "Highway"]

def is_street_name(elem):
    return (elem.attrib['k']=="addr:street")

def aduit_street_type(not_expected,street_name):
    m=street_type_re.search(street_name)
    if m:
        street_type=m.group()
        if street_type not in expected:
            not_expected[street_type].add(street_name)
    
def is_postcode(elem):
    return (elem.attrib['k']=="addr:postcode")

def aduit_zipcode(not_expected,postcode):
    if len(postcode)>5:
        not_expected = postcode
        return not_expected
    
def aduit_tag(filename,loc,test,aduit):
    not_expected=defaultdict(set)
    for i, elem in enumerate(get_element(filename)):
        if elem.tag==loc:
            for tag in elem.iter('tag'):
                if test(tag):
                    aduit(not_expected, tag.attrib['v'])
                    
    pprint.pprint(dict(not_expected))   
    


# update data
After we aduit the data, we check the display and change the data to what we expect.

In [14]:
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road",
            "Dr" : "Drive",
            "sq": "Square",
            "Sq": "Square",
            "Blvd": "Boulevard",
            "Boulvevard": "Boulevard",
            "Rd": "Road",
            "Express Way": "Expressway",
            "ave": "Avenue",
            "street": "Street",
            "Ct": "Court",
            "Hwy": "Highway",
            "Pkwy": "Parkway"
            }
audited_file = "osm_file_audit.osm"
def update_name(name, mapping):


    m = street_type_re.search(name)
    if m:
        street_type = m.group()

        if street_type not in expected:
            if street_type in mapping:
                name = re.sub(street_type_re, mapping[street_type], name)
                
    return name


#'CA': set(['Zanker Rd., San Jose, CA', 'Zanker Road, San Jose, CA'])
def street_only(name):
    s = street_type_re.search(name)
    if s:
        street_type = s.group()
        if street_type == "CA":
            name =street_name_only_re.match(name).group(0)
            return name
        if not street_type.isalpha():
            if "Ste" or "Suite"  in name:
                name=no_number_re.findall(name)[0]
                return name
    return name        

def update_postcode(postcode):
    postcode=re.sub("\D*", "",postcode)
    if len(postcode)>5:

        postcode=zipcode_re.search(postcode).group(0)
        return postcode
    return postcode



def update_tag(filename):

    
    
    with open(audited_file, 'wb') as output:
        output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        output.write('<osm>\n  ')    


        for i, elem in enumerate(get_element(filename)):
            if  elem.tag=="way" or elem.tag=="node" or elem.tag=="relation":
                for tag in elem.iter('tag'):
                    if is_street_name(tag):
                        tag.attrib['v']= street_only(tag.attrib['v'])
                        tag.attrib['v'] = update_name(tag.attrib['v'], mapping)
                    
                    if is_postcode(tag):
                        tag.attrib['v'] = update_postcode(tag.attrib['v'])
                        
            output.write(ET.tostring(elem, encoding='utf-8'))

        output.write('</osm>')                



In [15]:

def update_postcode(postcode):
    postcode=re.sub("\D*", "",postcode)
    if len(postcode)>5:
        
        print postcode

        postcode=zipcode_re.search(postcode).group(0)
        return postcode
    print type(postcode)
    return postcode

### when I try to clean up the data for postcode , I find out some user put city name in postcode field, so I have to check for postcode and replace the city name with a empty string. Another problem I find in postcode field is that some postcode is longer than 5 digits which is zipcode + indicate. I do not want the indicate, so I took the first five digits of those postcode.

### when I clean up the data for street type, I do not want abbreviation, so I update all the abbreviation. I also update the street type to only begin with a capital letter. At the same time, I fixed the wrong spelling. 

### after clean up the street type, I notice some street name is including suit number or the whole address. I took the street name out from the data and then update it as the street name

In [16]:
aduit_tag(audited_file,'way',is_street_name,aduit_street_type)
aduit_tag(audited_file,'relation',is_postcode ,aduit_zipcode)

{'680': set(['Ala 680']),
 'Alameda': set(['The Alameda']),
 'Barcelona': set(['Calle de Barcelona']),
 'Bascom': set(['S. Bascom']),
 'Cir': set(['Celadon Cir']),
 'East': set(['Park Circle East', 'Vanderbilt Court East']),
 'Esquela': set(['Camina Esquela']),
 'Ln': set(['Gaundabert Ln']),
 'Loop': set(['Infinite Loop', 'Village View Loop']),
 'Luna': set(['Calle de Luna']),
 'Madrid': set(['Corte de Madrid']),
 'Marino': set(['Via San Marino']),
 'Napoli': set(['Via Napoli']),
 'Palamos': set(['Via Palamos']),
 'Paviso': set(['Via Paviso']),
 'Plaza': set(['Rivermark Plaza']),
 'Portofino': set(['Via Portofino']),
 'Presada': set(['Paseo Presada']),
 'Seville': set(['Corte de Seville']),
 'Sorrento': set(['Via Sorrento']),
 'Volante': set(['Via Volante']),
 'West': set(['Park Circle West', 'Vanderbilt Court West']),
 'robles': set(['rio robles'])}
{}


In [17]:
aduit_tag(audited_file,'node',is_street_name,aduit_street_type)
aduit_tag(audited_file,'node',is_postcode ,aduit_zipcode)

{'17': set(['Hwy 17']),
 'Alameda': set(['The Alameda']),
 'Bellomy': set(['Bellomy']),
 'East': set(['Park Circle East', 'Rio Robles East']),
 'Flores': set(['Terreno De Flores']),
 'Franklin': set(['Franklin']),
 'Hamilton': set(['Mount Hamilton']),
 'Hill': set(['Blossom Hill']),
 'Julian': set(['West Julian']),
 'Loop': set(['Infinite Loop']),
 'Oro': set(['Via del Oro']),
 'Pkwy': set(['Great America Pkwy']),
 'Plaza': set(['Portal Plaza', 'Rivermark Plaza']),
 'Presada': set(['Paseo Presada']),
 'Row': set(['Santana Row']),
 'Sorrento': set(['Via Sorrento']),
 'Volante': set(['Via Volante']),
 'Walk': set(['Paseo de San Antonio Walk']),
 'West': set(['Park Circle West']),
 'Winchester': set(['Winchester'])}
{}


In [18]:
aduit_tag(audited_file,'relation',is_street_name,aduit_street_type)
aduit_tag(audited_file,'relation',is_postcode ,aduit_zipcode)

{'Paviso': set(['Via Paviso'])}
{}


# Grouping data 
I group the data in 5 different field for later use in SQL. 

In [19]:
NODE_FIELDS = ['id', 'user', 'uid', 'version','lat','lon', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

def shape_element(element):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  


    if element.tag == 'node':
        for a in element.attrib:
            if a in NODE_FIELDS:
                if a not in node_attribs:

                    node_attribs[a]=element.attrib[a]
        
     

        for tag in element.iter('tag'):
            if not PROBLEMCHARS.search(tag.attrib['k']):
                
                    
                each_tag={
                            "id":None,
                            "key":None,
                            "value":None,
                            "type":None
                }
                each_tag['id']=node_attribs['id']
                each_tag['value']=tag.attrib['v']
                if LOWER_COLON.search(tag.attrib['k']):
                    
                    sub_attrib=tag.attrib['k'].split(':',1)
                        
                    each_tag['key']=sub_attrib[1]

                    each_tag['type']=sub_attrib[0]
                else:
                    each_tag['key']=tag.attrib['k']
                    each_tag['type']='regular'
                    
        
                
                

                tags.append(each_tag)

        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for a in element.attrib:
            if a in WAY_FIELDS:
                if a not in way_attribs:

                    way_attribs[a]=element.attrib[a]
        count=0
        for nd in element.iter('nd'):
            
            each_way_node={}
            
            
            each_way_node['position']=count
            count+=1
            each_way_node['node_id']=nd.attrib['ref']
            each_way_node['id']=way_attribs['id']
            way_nodes.append(each_way_node)

        
        for tag in element.iter('tag'):
            if not PROBLEMCHARS.search(tag.attrib['k']):
                
                    
                each_tag={}
                each_tag['id']=way_attribs['id']
                each_tag['value']=tag.attrib['v']
                if LOWER_COLON.search(tag.attrib['k']):
                    
                    sub_attrib=tag.attrib['k'].split(':',1)
                        
                    each_tag['key']=sub_attrib[1]

                    each_tag['type']=sub_attrib[0]
                else:
                    each_tag['key']=tag.attrib['k']
                    each_tag['type']='regular'
                    
        
                tags.append(each_tag)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# write data into csv file


In [20]:
def process_map(file_in):
    """Iteratively process each XML element and write to csv(s)"""
    data = []
    with codecs.open(NODES_PATH, 'w') as nodes_file,\
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()
        
        
        for element in get_element(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
        return data

### preview of the data.

In [21]:
data = process_map(audited_file)
pprint.pprint(data[0:6])

[{'node': {'changeset': '11686320',
           'id': '25457954',
           'lat': '37.1582245',
           'lon': '-121.6574737',
           'timestamp': '2012-05-24T03:24:59Z',
           'uid': '14293',
           'user': 'KindredCoda',
           'version': '10'},
  'node_tags': []},
 {'node': {'changeset': '11721513',
           'id': '25457955',
           'lat': '37.1648535',
           'lon': '-121.6653013',
           'timestamp': '2012-05-27T23:43:57Z',
           'uid': '14293',
           'user': 'KindredCoda',
           'version': '10'},
  'node_tags': []},
 {'node': {'changeset': '11721513',
           'id': '25457957',
           'lat': '37.186653',
           'lon': '-121.6871419',
           'timestamp': '2012-05-27T23:43:57Z',
           'uid': '14293',
           'user': 'KindredCoda',
           'version': '15'},
  'node_tags': []},
 {'node': {'changeset': '11721596',
           'id': '25457960',
           'lat': '37.2273856',
           'lon': '-121.7433767',
   